In [ ]:
import Geant4 as g4
from G4Constants import *
import numpy as np
from math import pi

class iZIP5_Construction(g4.G4VUserDetectorConstruction):
	def __init__(self):
		super().__init__()
		self.world_phys = None  # Store to prevent gc

	def Construct(self):
		# World volume
		worldSolid = g4.G4Box("WorldSolid", 1*m, 1*m, 1*m)
		worldMaterial = g4.G4Material.GetMaterial("G4_AIR")
		worldLogical = g4.G4LogicalVolume(worldSolid, worldMaterial, "WorldLogical")
		self.world_phys = g4.G4PVPlacement(None, g4.G4ThreeVector(), worldLogical, "World", None, False, 0)

		# Construct iZIP5 solid
		ZipRad = 38.1*mm
		ZipThick = 25.4*mm - 1.2*um
		ZipAxis1Len = 75.4888*mm
		ZipAxis2Len = 72.1868*mm

		# Diagonal Flat
		ExtraFlats = g4.G4ThreeVector()
		ExtraFlats.setRhoPhiZ(37.7444*mm, 45.*deg, 0.)

		# Start Cylindrical Crystal
		zipSolid = g4.G4Tubs('iZIP5', 0, ZipRad, ZipThick/2., 0, 2*pi)
		print(zipSolid)

		# Both side cuts
		tol = 1.*nm
		zipCutBox = g4.G4Box("ZipCutBox", ZipAxis1Len/2., ZipAxis2Len/2., (ZipThick + tol)/2.)
		zipSolid = g4.G4IntersectionSolid('iZIP5', zipSolid, zipCutBox)
		print(zipCutBox)

		# Remove additioanl single flats
		phi = ExtraFlats.phi()
		rflat = ExtraFlats.r()

		if rflat < ZipRad:
			delta = ZipRad - rflat
			hwid  = np.sqrt(ZipRad*ZipRad - rflat*rflat)

			# Make box with double the sagitta wide, and double the chord long
			flatCut = g4.G4Box("FlatCut", delta, hwid, (ZipThick + tol)/2.)
			print(flatCut)
			pos = g4.G4ThreeVector()
			pos.setRhoPhiZ(ZipRad, phi, 0.)

			rotation = g4.G4RotationMatrix()
			rotation.rotateZ(-phi)
			zipSolid = g4.G4SubtractionSolid('iZIP5', zipSolid, flatCut, rotation, pos)

		# Create logical and place
		zipMaterial = g4.G4Material.GetMaterial("G4_Ge")
		zipLogical = g4.G4LogicalVolume(zipSolid, zipMaterial, "Detector")
		g4.G4PVPlacement(g4.G4RotationMatrix(), g4.G4ThreeVector(), zipLogical, "Detector", worldLogical, False, 0)

		return self.world_phys


class MyDetectorConstruction(g4.G4VUserDetectorConstruction):
    def __init__(self):
        super().__init__()
        self.world_phys = None  # Store reference to prevent it from being garbage collected

    def Construct(self):
        # Define materials
        nist = g4.G4NistManager.Instance()
        air = nist.FindOrBuildMaterial("G4_AIR")
        silicon = nist.FindOrBuildMaterial("G4_Si")

        # World volume
        world_solid = g4.G4Box("World", 1*g4.m, 1*g4.m, 1*g4.m)
        world_logic = g4.G4LogicalVolume(world_solid, air, "World")
        self.world_phys = g4.G4PVPlacement(None, g4.G4ThreeVector(), world_logic, "World", None, False, 0)

        # Detector volume
        detector_solid = g4.G4Box("Detector", 5*g4.cm, 5*g4.cm, 1*g4.cm)
        detector_logic = g4.G4LogicalVolume(detector_solid, silicon, "Detector")
        g4.G4PVPlacement(None, g4.G4ThreeVector(0, 0, 0), detector_logic, "Detector", world_logic, False, 0)

        return self.world_phys  # Ensure the world volume is returned properly